In [1]:
# import necessary libaries for the KDD 99 challenge
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif

%matplotlib inline

In [2]:
# data_full = pd.read_csv("kdd99/kddcup.data")

# manually add the column names
cols = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
data_ten = pd.read_csv("kdd99/kddcup.data_10_percent", header=None, names=cols)

# describe the data
data_ten.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494021.000000,4.940210e+05,4.940210e+05,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,...,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000
mean,47.979302,3.025610e+03,8.685324e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148247,0.010212,...,232.470778,188.665670,0.753780,0.030906,0.601935,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.746472,9.882181e+05,3.304000e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355345,1.798326,...,64.745380,106.040437,0.410781,0.109259,0.481309,0.042133,0.380593,0.380919,0.230590,0.230140
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
# Get an idea what kind of attacks are in the data set
data_ten["label"].value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [4]:
# partition into: normal/abnormal
data_ten_c = data_ten.copy(deep=True)
data_ten_c["label"][data_ten_c["label"] != "normal."] = "intrusion"

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
# convert to numeric, using a label encoder on those columes whose data type is an object 
# (e.g. string, and not int or float)

for c in data_ten_c.columns:
    if data_ten_c.dtypes[c] == "object":
        le = LabelEncoder()
        data_ten_c[c] = le.fit_transform(data_ten_c[c])
        
# take a look at the new dataframe, were cardinal values such as "service" and "flag" have been encoded with ints        
data_ten_c.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,22,9,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,1
1,0,1,22,9,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,1
2,0,1,22,9,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1
3,0,1,22,9,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1
4,0,1,22,9,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,1


In [6]:
# remove features with zero variance (e.g. constants)
# Don't use sklearn's VarianceThreshold because we want to keep working with DataFrames, not Numpy arrays
mask = ~np.isclose(0, data_ten_c.var())
data_ten_c = data_ten_c.loc[:, mask]
cols = [cols[i] for i in range(len(cols) - 1) if mask[i]] 

In [7]:
# Min Max Scaling
mm = MinMaxScaler()
scaled = mm.fit_transform(data_ten_c.values)
data_ten_c = pd.DataFrame(scaled, columns=data_ten_c.columns)

# Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(data_ten_c.drop(["label"], axis=1), \
                                                    data_ten_c["label"], test_size=0.2)
X_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
347743,0.0,1.0,0.692308,0.9,1.514331e-07,0.000028,0.0,0.0,0.0,0.0,...,1.000000,0.956863,0.96,0.01,0.01,0.00,0.0,0.0,0.0,0.0
479692,0.0,0.5,0.692308,0.1,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,...,1.000000,0.023529,0.02,0.07,0.00,0.00,0.0,0.0,1.0,1.0
11071,0.0,0.0,0.215385,0.9,1.488371e-06,0.000000,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0
32309,0.0,0.5,0.338462,0.9,2.350818e-07,0.001688,0.0,0.0,0.0,0.0,...,0.011765,1.000000,1.00,0.00,0.33,0.02,0.0,0.0,0.0,0.0
172608,0.0,0.0,0.215385,0.9,1.488371e-06,0.000000,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0


In [8]:
# Select K best
sel = SelectKBest(chi2, k=12)
sel.fit(X_train, Y_train)

# remove "label" from cols if present
try:
    cols.remove("label")
except:
    pass

# print the score values. SelectKBest selects the features with the LOWEST score!!
# ANOVA: H_0 is: samples x_i and z_i come from common distribution => thus, if i know value of x_1, i may know z_1
# scores are better if greater, p-values are better if smaller
# The p-value of a feature selection score indicates the probability that this score or a higher score would be obtained if this variable showed no interaction with the target.
scores = sel.scores_
r = pd.DataFrame(scores, index=cols, columns=["KBestScore"])
# r.sort_values(ascending="True", by="KBestScore")
r["mask"] = sel.get_support()
r = r.sort_values("KBestScore")
r

,KBestScore,mask
src_bytes,0.201623,False
num_failed_logins,0.415142,False
dst_host_rerror_rate,0.560846,False
urgent,1.716582,False
land,2.267954,False
num_compromised,4.168807,False
dst_host_srv_rerror_rate,4.730121,False
srv_rerror_rate,4.803466,False
rerror_rate,4.820236,False
hot,11.758518,False


In [11]:
data_ten_c["count"].value_counts()

1.000000    227895
0.001957     39214
0.998043     26598
0.003914     11219
0.005871      5812
0.996086      5605
0.007828      5400
0.009785      4469
0.011742      3708
0.013699      3254
0.015656      2921
0.017613      2692
0.019569      2465
0.021526      2291
0.023483      2013
0.025440      1832
0.027397      1706
0.029354      1466
0.994129      1426
0.031311      1378
0.033268      1224
0.035225      1123
0.201566      1037
0.037182      1021
0.219178       994
0.199609       964
0.258317       963
0.211350       959
0.238748       955
0.221135       951
             ...  
0.753425         1
0.757339         1
0.653620         1
0.788650         1
0.784736         1
0.602740         1
0.737769         1
0.796477         1
0.776908         1
0.696673         1
0.786693         1
0.675147         1
0.778865         1
0.628180         1
0.637965         1
0.600783         1
0.810176         1
0.763209         1
0.708415         1
0.731898         1
0.636008         1
0.804305    